In [10]:
from pyomo.environ import *


In [11]:
def build_model(yields):
    model = ConcreteModel()

    # Variables
    model.X = Var(["WHEAT", "CORN", "BEETS"], within=NonNegativeReals)
    model.Y = Var(["WHEAT", "CORN"], within=NonNegativeReals)
    model.W = Var(
        ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"],
        within=NonNegativeReals,
    )

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST = 238 * model.Y["WHEAT"] + 210 * model.Y["CORN"]
    model.SALES_REVENUE = (
        170 * model.W["WHEAT"] + 150 * model.W["CORN"]
        + 36 * model.W["BEETS_FAVORABLE"] + 10 * model.W["BEETS_UNFAVORABLE"]
    )
    model.OBJ = Objective(
        expr=model.PLANTING_COST + model.PURCHASE_COST - model.SALES_REVENUE,
        sense=minimize
    )

    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(
        yields[0] * model.X["WHEAT"] + model.Y["WHEAT"] - model.W["WHEAT"] >= 200
    )
    model.CONSTR.add(
        yields[1] * model.X["CORN"] + model.Y["CORN"] - model.W["CORN"] >= 240
    )
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE"] - model.W["BEETS_UNFAVORABLE"] >= 0
    )
    model.W["BEETS_FAVORABLE"].setub(6000)
    
    model.CONSTR.add(model.X["WHEAT"] == 120)
    model.CONSTR.add(model.X["CORN"] == 80)
    model.CONSTR.add(model.X["BEETS"] == 300)

    return model

In [12]:
avg_yields = [2.5, 3, 20]
avg_model = build_model(avg_yields)
avg_solver = SolverFactory("gurobi")
avg_solver.solve(avg_model)

{'Problem': [{'Name': 'x1', 'Lower bound': -118600.0, 'Upper bound': -118600.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 9, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0019998550415039062', 'Error rc': 0, 'Time': 0.14982223510742188}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
bel_yields = [2, 2.4, 16]
bel_model = build_model(bel_yields)
bel_solver = SolverFactory("gurobi")
bel_solver.solve(bel_model)

{'Problem': [{'Name': 'x1', 'Lower bound': -55120.0, 'Upper bound': -55120.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 9, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0009999275207519531', 'Error rc': 0, 'Time': 0.14844703674316406}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
abo_yields = [3, 3.6, 24]
abo_model = build_model(abo_yields)
abo_solver = SolverFactory("gurobi")
abo_solver.solve(abo_model)

{'Problem': [{'Name': 'x1', 'Lower bound': -148000.0, 'Upper bound': -148000.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 9, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.15018343925476074}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
print(f"{value(bel_model.OBJ):.1f}")

-55120.0


In [16]:
print(f"{value(avg_model.OBJ):.1f}")

-118600.0


In [17]:
print(f"{value(abo_model.OBJ):.1f}")

-148000.0


In [18]:
print(f"{(value(bel_model.OBJ) + value(avg_model.OBJ) + value(abo_model.OBJ))/3:.3f}")

-107240.000
